#Dynamic Time Warping#
In this notebook, you are going to classify time series with the 1-NN algorithm, usign two different approaches to compute the distance between time series: the Euclidean distance and the Dynamic Time Warping (DTW) distance. The comparison will be made  for time series of equal length as well as for varying-length time series.

##Processing the data##
The goal is to predict, based on hourly rentals, if a given day is a working day or not. Start by loading the "hours.csv" file, where one line contains information about the bike renting system for one hour. Take care to properly parse the date information of the data as done before. The number of rentals is recorded in the column 'cnt'.

In [1]:
import pandas as pd

data = pd.read_csv('hour.csv', parse_dates=['dteday'])
print(data.shape)
data.head(10)

(17379, 17)


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,7,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,8,2011-01-01,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,9,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,10,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


We want to operate on days, not on hours, but we need to keep track of the hourly data, as the sequences of hourly rentals will be our time-series. The other variables are not necessary. 
Find a way to aggregate the hourly observations, and create a dataframe with two columns: **'counts'** and **'workingday'**. The former should contain a list of the hourly counts. The latter should contain 0's or 1's indicating whether a given row correspond to a working day or not (0 = no, 1 = yes).
Note that your lists should contain exactly 24 elements.

In [2]:
ts = data.groupby(data['dteday'].dt.date)['cnt'].agg(lambda x: list(x)).to_frame('counts'); # aggregate observations from the same day by creating a list of the hourly counts
labels = data.groupby(data['dteday'].dt.date)['workingday'].agg('mean').to_frame() # 1 = working day, 0 = not working day ; the mean is a convenient way to reduce 24 1's or 0's to a single 1 or 0

ts['workingday'] = labels # bundle everything in one dataframe 
ts.head(20)


,counts,workingday
dteday,,
2011-01-01,"[16, 40, 32, 13, 1, 1, 2, 3, 8, 14, 36, 56, 84...",0
2011-01-02,"[17, 17, 9, 6, 3, 2, 1, 8, 20, 53, 70, 93, 75,...",0
2011-01-03,"[5, 2, 1, 3, 30, 64, 154, 88, 44, 51, 61, 61, ...",1
2011-01-04,"[5, 2, 1, 2, 4, 36, 94, 179, 100, 42, 57, 78, ...",1
2011-01-05,"[6, 6, 2, 2, 3, 33, 88, 195, 115, 57, 46, 79, ...",1
2011-01-06,"[11, 4, 2, 1, 4, 36, 95, 219, 122, 45, 59, 84,...",1
2011-01-07,"[17, 7, 1, 1, 5, 34, 84, 210, 134, 63, 67, 59,...",1
2011-01-08,"[25, 16, 16, 7, 1, 5, 2, 9, 15, 20, 61, 62, 98...",0
2011-01-09,"[25, 12, 11, 4, 1, 1, 1, 6, 10, 19, 49, 49, 83...",0


Now that your data is in the right format, use the **train_test_split** method of the **sklearn.cross_validation** module to split it in a training set (66% of the data) and test set (33% of the data). Make sure the shapes of the returned data structures make sense. 

In [3]:
from sklearn.cross_validation import train_test_split

# Fill in the line bewlo to obtain train and test sets from your initial data
X_train, X_test, y_train, y_test = train_test_split(ts['counts'], ts['workingday'], test_size=0.33, random_state=42)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape
# Feel free to add other sanity checks ; for example, there is one obvious relationship between 
# the shapes of the X and y DataFrames that you might want to check with an assert()


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-3-081467797062>, line 5)

##Implementing the algorithms##
To perform the desired tasks, we need to implement several things:
- A function to compute the Euclidean distance between two time series
- A function to compute the DTW distance between two time series
- A function to classify a time series according to its nearest neighbor, using a arbitrary distance function
- A wrapper function to run our 1NN implementation on the whole test set and compute the accuracy of the approach

Start by defining a method that, given two time series, return the Euclidean distance between them. The formula for the Euclidean distance $d$ between two time-series $a$ and $b$ of length $n$ is the following:
$$d = \sqrt{\sum_{i=1}^n (a_i - b_i)^2}$$

In [ ]:
import math

def euclid_dist(s1,s2): # return euclidean distance between two time-series 
    sqdiffs = [(a_i - b_i)**2 for a_i, b_i in zip(s1, s2)]
    return math.sqrt(sum(sqdiffs))

Now define a method that classifies one time series using the 1 Nearest Neighbor algorithm. This method takes 4 arguments:
- X_train: the time series of the training set
- y_train: the corresponding labels of the time series (working day or not)
- test_s: the instance to classify
- distance: the distance function to use (for the moment we only have the Euclidean distance available)
The returned value should be the prediction for the test instance, 0 or 1.

In [ ]:
def oneNearestNeighbor(X_train,y_train,test_s,distance): # classifie the test_s time series using 1NN ; the distance is computed using the provided distance function
    min_dist = float('inf')
    prediction = -1
    for i in range(len(y_train)):
        d = distance(X_train[i],test_s)
        if d < min_dist:
            min_dist = d
            prediction = y_train[i]
    return prediction

Define a method that will run your **oneNearestNeighbor** on all the instances of the test set and return the classification accuracy. The method takes 5 arguments:
- X_train: the time series of the training set
- y_train: the corresponding labels of the time series (working day or not)
- X_test and y_test: same, but for the test set
- distance: the distance function to use 

In [ ]:
def classify(X_train,y_train,X_test,y_test,distance): # classify all the instances of the test set using 1NN with the provided distance function
    correct = 0.0
    for j in range(len(y_test)):
        prediction = oneNearestNeighbor(X_train,y_train,X_test[j],distance)
        if prediction == y_test[j]:
            correct += 1

    accuracy = correct/len(y_test)
    return accuracy

Now use your methods to classify the test instances using the Euclidean distance. Is the performance good? What would be the performance of a baseline classifier which always predicts the majority class?

In [76]:
accuracy = classify(X_train,y_train,X_test,y_test,euclid_dist)
print accuracy

0.855371900826


The next cell provides an implementation of the DTW distance. Take some time to understand the code and try to match what it does to what you have seen of dynatmic time warping in class. 

In [ ]:
def DTWDistance(s1, s2): # returns the DTW distance between two time series s1 and s2
    DTW={}

    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return math.sqrt(DTW[len(s1)-1, len(s2)-1])

Run your **classify** method again, this time using the DTWDistance. Is the performance better? 

In [77]:
accuracy = classify(X_train,y_train,X_test,y_test,DTWDistance) # should take ~3 min to run
print accuracy

0.971074380165


So far, all the time series had the same length (24). Let's change that, by arbitrarily removing the hourly counts smaller than 50. The next cell create a new dataframe with varying-length time series.

In [79]:
def trim(row):  # 'trim' a time series by removing elements from it
    tmp = []
    for c in row.counts:
        if c > 50:
            tmp.append(c)
    row.counts = tmp
    return row

varts = ts.apply(trim,axis=1) # apply our trim method on all rows of the ts datarame
varts.head()

,counts,workingday
2011-01-01,"[56, 84, 94, 106, 110, 93, 67]",0
2011-01-02,"[53, 70, 93, 75, 59, 74, 76, 65, 53]",0
2011-01-03,"[64, 154, 88, 51, 61, 61, 77, 72, 76, 157, 157...",1
2011-01-04,"[94, 179, 100, 57, 78, 97, 63, 65, 83, 212, 18...",1
2011-01-05,"[88, 195, 115, 57, 79, 71, 62, 62, 89, 190, 16...",1


In the next cell, we re-create our X and y matrices. This time, they contain time series of varying lengths.

In [80]:
X_train, X_test, y_train, y_test = train_test_split(varts['counts'], varts['workingday'], test_size=0.33, random_state=42)

Finally, we compare the two distances, this time on the varying-length time series dataset. Do you notice any significant change in performance?

In [81]:
euclid_accuracy = classify(X_train,y_train,X_test,y_test,euclid_dist)
DTW_accuracy = classify(X_train,y_train,X_test,y_test,DTWDistance) 
print euclid_accuracy
print DTW_accuracy

0.173553719008
0.979338842975
